In [ ]:
import sys
sys.path.append("..")
sys.path.append("../../context-mediation")

In [ ]:
!nvidia-smi

In [ ]:
import transformers

device = "cuda"
config = "EleutherAI/gpt-j-6B"

model = transformers.AutoModelForCausalLM.from_pretrained(config, revision="float16", low_cpu_mem_usage=True)
model.to(device)

tokenizer = transformers.AutoTokenizer.from_pretrained(config)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
from src import data

counterfact = data.load_dataset("counterfact", split="train")

In [ ]:
counterfact[0]

In [ ]:
from collections import defaultdict

samples_by_relation = defaultdict(list)
for sample in counterfact:
    relation_id = sample["source"]["requested_rewrite"]["relation_id"]
    samples_by_relation[relation_id].append(sample)

In [ ]:
len(samples_by_relation), {r: len(s) for r, s in samples_by_relation.items()}

In [ ]:
from src.utils import tokenizer_utils

import baukit
import torch
from torch.utils.data import DataLoader
from tqdm.auto import tqdm

preds_by_relation = defaultdict(list)
logp_by_relation = defaultdict(list)
with counterfact.formatted_as("torch"):
    loader = DataLoader(counterfact, batch_size=32, shuffle=False)
    for batch in tqdm(loader):
        inputs = tokenizer(
            batch["prompt"],
            return_tensors="pt",
            padding="longest",
        ).to(device)
        with torch.inference_mode():
            outputs = model(**inputs)

        batch_idx = torch.arange(len(batch["prompt"]))
        prompt_idx = inputs.attention_mask.sum(dim=-1) - 1
        topk = torch.log_softmax(outputs.logits, dim=-1)[batch_idx, prompt_idx].topk(dim=-1, k=5)
        ids = topk.indices
        logps = topk.values
        tokens = tokenizer_utils.batch_convert_ids_to_tokens(ids.tolist(), tokenizer)

        for rid, preds, logp in zip(
            batch["source"]["requested_rewrite"]["relation_id"],
            tokens,
            logps.cpu(),
        ):
            preds_by_relation[rid].append(preds)
            logp_by_relation[rid].append(logp)

In [ ]:
preds_by_relation["P264"][0]

In [ ]:
from collections import defaultdict

import estimate

layer = 15
n_train = 5
n_test = 50


relation_templates = {
    "P103": "{} speaks the language of",
    "P140": "{} follows the religion of",
    "P740": "{} originates from",
    "P190": "{} has a sister city named \"",
    "P178": "{} was originally developed by",
    "P176": "{} is developed by the company",
    "P413": "In their sport, {} plays the position of",
    "P39": "{} has the occupation of",
    "P407": "{} was written in the language of",
    "P101": "{} works in the field of",
    "P1412": "{} speaks the language of",
    "P27": "{} is originally from the country of",
    "P106": "{} has the occupation of",
    "P136": "{} is most commonly associated with the genre of",
    "P159": "{} is based in the city of",
    "P276": "{} is located in the city of",
}


accs_by_relation = defaultdict(dict)
for rid, samples in samples_by_relation.items():
    print(f"---- {rid} ----")

    # Only use examples for which the model encodes the correct relation.
    samples = [
        s
        for i, s in sorted(
            enumerate(samples),
            key=lambda x: logp_by_relation[rid][x[0]][0].item(),
            reverse=True,
        )
        if any(
            pred.strip("ĠĊ ").lower()
            in
            s["source"]["requested_rewrite"]["target_true"]["str"].lower()
            for pred in preds_by_relation[rid][i][:1]
            if pred.strip("ĠĊ ").lower()
        )
    ]
    print(f"{len(samples)} known samples")

    # When picking training examples, choose a diverse set of labels.
    trains = []
    tests = []
    seen = set()
    for sample in samples:
        label = sample["source"]["requested_rewrite"]["target_true"]["str"]
        if len(trains) >= n_train:
            if len(tests) < n_test:
                tests.append(sample)
            continue
        elif label not in seen:
            trains.append(sample)
            seen.add(label)
        else:
            tests.append(sample)

    # Pick best relation text via heuristic
    if rid in relation_templates:
        relation = relation_templates[rid]
    else:
        #         continue
        # Pick by heuristic
        rs0 = [
            sample["source"]["requested_rewrite"]["prompt"]
            for sample in samples
        ]
        # Always prefer one that puts the subject first:
        rs1 = [r for r in rs0 if r.startswith("{}")]
        # Then, prefer one with no special punctuation:
        rs2 = [r for r in rs1 if not any(x in r for x in "?,:;.")]
        # Then prefer the longest one:
        rs3 = sorted(rs2, key=lambda r: len(r), reverse=True)
        relation = None
        for rs in (rs3, rs2, rs1, rs0):
            if rs:
                relation = rs[0]
                break
        assert relation is not None

    batch = [
        (
            train["source"]["requested_rewrite"]["subject"],
            train["source"]["requested_rewrite"]["target_true"]["str"]
        )
        for train in trains
    ]
    print(relation)
    print(batch)
    print(len(tests), "test examples")
#     continue
#     print([t["source"]["requested_rewrite"]["prompt"] for t in trains])
#     continue

    operator_a, _ = estimate.relation_operator_from_batch(
        model=model,
        tokenizer=tokenizer,
        relation=relation,
        samples=batch,
        layer=layer,
        device=device,
    )

    correct_by_k = defaultdict(int)
    for test in tqdm(tests, desc="test"):
        rr = test["source"]["requested_rewrite"]
        subject = rr["subject"]
        expected = rr["target_true"]["str"]

        preds = operator_a(subject, device=device, return_top_k=5)
        for k in (1, 3, 5):
            actuals = [p.strip("ĠĊ ").lower() for p in preds[:k]]
            correct_by_k[k] += any(
                expected.lower().strip().startswith(actual)
                for actual in actuals
            )

    for k in (1, 3, 5):
        accuracy = correct_by_k[k] / len(tests)
        print(f"top-{k} accuracy: {accuracy:.2f}")
        accs_by_relation[rid][k] = accuracy

In [ ]:
import matplotlib.pyplot as plt

plt.

In [ ]:
targets_by_relation = {}
for rid, samples in samples_by_relation.items():
    targets_by_relation[rid] = {
        sample["source"]["requested_rewrite"]["target_true"]["str"]
        for sample in samples
    }
{rid: len(targets) for rid, targets in targets_by_relation.items()}

In [ ]:
prompts_by_relation = {}
for rid, samples in samples_by_relation.items():
    prompts_by_relation[rid] = [
        sample["source"]["requested_rewrite"]["prompt"]
        for sample in samples
    ]
{rid: prompts[0] for rid, prompts in prompts_by_relation.items()}